In [1]:
### to perform double segmentation, a standard and an ROI model is needed
### we will use the previously trained standard deeplab example and ROI deeplab example

In [8]:
import torch

import general_config
import constants
import numpy as np
import save_complete_nii_results
from utils import inference
from utils import prepare_models
from utils.training_utils.box_utils import convert_offsets_to_bboxes, wh2corners
from utils.training_utils import training_processing
from utils.ROI_crop import roi_crop
from utils.ROI_crop import predict_ROI
from utils.training_utils import prints, training_setup
from utils import metrics
from utils import visualization


def get_double_seg_predictions(model_a, model_b, dataset, logsoft_preds=False):
    """
    Args:
    model_a: str -> model to extract ROI (standard)
    model_b: str -> model to predict from ROI
    dataset: str
    """
    models, params_list, configs_list = prepare_models.prepare([model_a, model_b], dataset)
    model_a, model_b = models
    validation_loader = training_setup.prepare_val_loader(params_list[0], configs_list[0])
    validation_loader_2 = training_setup.prepare_val_loader(params_list[1], configs_list[1])
    
    coords_n_scores = predict_ROI.get_segmentor_pred_coords(validation_loader, model_a, params_list[0], configs_list[0])
    _, predictions_double_seg = predict_ROI.segment_with_computed_ROI(validation_loader_2, model_b, params_list[1], configs_list[1], coords_n_scores, logsoft_preds=logsoft_preds)
    
    return predictions_double_seg, validation_loader_2

In [9]:
# perform double segmentation, add err_margin to standard model params.json
_, _ = get_double_seg_predictions("Deeplab_standard_example", "Deeplab_ROI_example", "ACDC_training")

Constructing model:  Deeplab_standard_example
Number of classes:  4
Loading model from:  experiments/ACDC_training/Deeplab_standard_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Constructing model:  Deeplab_ROI_example
Number of classes:  4
Loading model from:  experiments/ACDC_training/Deeplab_ROI_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Dataset mean and std:  69.52275548950034 90.27520888722917
Dataset mean and std:  69.52275548950034 90.27520888722917
current volume dice:  [0.04534049 0.13396471 0.62595698] 0.26842072722062765
current volume dice:  [0.03338257 0.1592437  0.61338638] 0.2686708824114217
current volume dice:  [0.3992664  0.3710862  0.86990042] 0.5467510072855731
current volume dice:  [0.46345722 0.40900918 0.87872454] 0.5837303142162023
c

In [11]:
# double segmentation on properly trained models
_, _ = get_double_seg_predictions("ResNeXt_DeepLabV3_plus", "2D_Unet_5_20_get_min", "ACDC_training")

Constructing model:  ResNeXt_DeepLabV3_plus
Number of classes:  4
Loading model from:  experiments/ACDC_training/ResNeXt_DeepLabV3_plus/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Constructing model:  2D_Unet_5_20_get_min
Number of classes:  4
Loading model from:  experiments/ACDC_training/2D_Unet_5_20_get_min/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  4320132
Total number of trainable parameters of model:  4320132 

Dataset mean and std:  69.52275548950034 90.27520888722917
Using per slice normalization!
current volume dice:  [0.92374806 0.91100657 0.97581802] 0.9368575482996508
current volume dice:  [0.82859669 0.91844213 0.97091742] 0.9059854159142487
current volume dice:  [0.94790761 0.89493365 0.97688765] 0.9399096356886073
current volume dice:  [0.95597562 0.89736374 0.97719388] 0.943511078843866
current volume dice:  [0.95727067

In [ ]:
### Ensembling: softmax predictions of different models are summed and averaged

In [ ]:
# ensembling two standard models

In [12]:
# load the model weigths, again using the previously trained example models
models, params_list, configs_list = prepare_models.prepare(["Deeplab_standard_example", "2D_Unet_standard_example"], "ACDC_training")
model_a, model_b = models
validation_loader_a = training_setup.prepare_val_loader(params_list[0], configs_list[0])
validation_loader_b = training_setup.prepare_val_loader(params_list[1], configs_list[1])

Constructing model:  Deeplab_standard_example
Number of classes:  4
Loading model from:  experiments/ACDC_training/Deeplab_standard_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Constructing model:  2D_Unet_standard_example
Number of classes:  4
Loading model from:  experiments/ACDC_training/2D_Unet_standard_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  17266436
Total number of trainable parameters of model:  17266436 

Dataset mean and std:  69.52275548950034 90.27520888722917
Using per slice normalization!


In [13]:
# perform inference with both
predictions_deeplab, _ = inference.run_model_inference(validation_loader_a, model_a, params_list[0], logsoft_preds=True)
predictions_unet, _ = inference.run_model_inference(validation_loader_b, model_b, params_list[1], logsoft_preds=True)

In [14]:
# finally, ensemble standard models
logsoftmax_volumes = [predictions_unet, predictions_deeplab]
combined_volumes = inference.ensemble_inference(logsoftmax_volumes, validation_loader_a)

Current volume dice:  [0.20436628 0.44778516 0.82166628]
Current volume dice:  [0.19753577 0.52075295 0.79161636]
Current volume dice:  [0.65256617 0.65850826 0.92779247]
Current volume dice:  [0.60577065 0.66004335 0.91434846]
Current volume dice:  [0.41532532 0.52544064 0.86381365]
Current volume dice:  [0.42842557 0.45746139 0.82875264]
Current volume dice:  [0.07529821 0.22545713 0.47440489]
Current volume dice:  [0.10235131 0.21887417 0.42808399]
Current volume dice:  [0.76734804 0.65308529 0.87857483]
Current volume dice:  [0.56858307 0.56737305 0.6715438 ]
Current volume dice:  [0.46493029 0.64476615 0.73566642]
Current volume dice:  [0.13964435 0.44314926 0.59337748]
Current volume dice:  [0.7445153  0.62867091 0.83081775]
Current volume dice:  [0.46399761 0.66510109 0.68567197]
Current volume dice:  [0.62064557 0.54377767 0.81786826]
Current volume dice:  [0.28885135 0.39486866 0.66811594]
Current volume dice:  [0.7314107  0.67958594 0.9251508 ]
Current volume dice:  [0.687776

In [15]:
# it is also possible to ensemble standard and ROI models
# add logsoft flag
double_seg_preds, _ = get_double_seg_predictions("Deeplab_standard_example", "Deeplab_ROI_example", "ACDC_training", logsoft_preds=True)

Constructing model:  Deeplab_standard_example
Number of classes:  4
Loading model from:  experiments/ACDC_training/Deeplab_standard_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Constructing model:  Deeplab_ROI_example
Number of classes:  4
Loading model from:  experiments/ACDC_training/Deeplab_ROI_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Dataset mean and std:  69.52275548950034 90.27520888722917
Dataset mean and std:  69.52275548950034 90.27520888722917
current volume dice:  [0.04534049 0.13396471 0.62595698] 0.26842072722062765
current volume dice:  [0.03338257 0.1592437  0.61338638] 0.2686708824114217
current volume dice:  [0.3992664  0.3710862  0.86990042] 0.5467510072855731
current volume dice:  [0.46345722 0.40900918 0.87872454] 0.5837303142162023
c

In [16]:
# ensemble standard models and double segmentation output
logsoftmax_volumes = [predictions_unet, predictions_deeplab, double_seg_preds]
combined_volumes = inference.ensemble_inference(logsoftmax_volumes, validation_loader_a)

Current volume dice:  [0.02035941 0.20548403 0.77777321]
Current volume dice:  [0.07137064 0.26004728 0.76365315]
Current volume dice:  [0.55626845 0.49537819 0.91925573]
Current volume dice:  [0.55014691 0.54913464 0.92616884]
Current volume dice:  [0.20856772 0.18936229 0.82037619]
Current volume dice:  [0.27545524 0.21203892 0.80844108]
Current volume dice:  [0.04947614 0.29560457 0.61457536]
Current volume dice:  [0.02762908 0.32528386 0.6429376 ]
Current volume dice:  [0.70426673 0.49107474 0.86437899]
Current volume dice:  [0.56732363 0.4978361  0.70597043]
Current volume dice:  [0.47247539 0.53628094 0.68732482]
Current volume dice:  [0.23007968 0.33534874 0.50117096]
Current volume dice:  [0.72894559 0.5236912  0.8202036 ]
Current volume dice:  [0.55755976 0.62650459 0.79578223]
Current volume dice:  [0.60640472 0.5965079  0.84661856]
Current volume dice:  [0.23953262 0.44694045 0.7854128 ]
Current volume dice:  [0.71316542 0.64653552 0.93988534]
Current volume dice:  [0.664363

In [17]:
### TESTING

In [18]:
def get_double_seg_predictions_test(model_a, model_b, dataset):
    """
    Args:
    model_a: str -> model to extract ROI
    model_b: str -> model to predict from ROI
    dataset: str
    """
    coords_n_scores = get_test_coords(model_a, dataset)
    model_roi, params, config = prepare_models.prepare([model_b], dataset)
    test_loader = training_setup.prepare_test_loader(params, config)
    
    predictions_double_seg = predict_ROI.segment_with_computed_ROI_test(test_loader, model_roi, params, config, coords_n_scores, logsoft_preds=True)
    
    return predictions_double_seg

def get_test_coords(model, dataset):
    model, params, config = prepare_models.prepare([model], dataset)
    test_loader = training_setup.prepare_test_loader(params, config)
    coords_n_scores = predict_ROI.get_segmentor_pred_coords(test_loader, model, params, config)
    return coords_n_scores

In [19]:
# the test set is in a directory of its own, so copy/move the previously trained models' experiments folders
# in experiments/acdc_test_set
# also, don't forget to change the dataset name from ACDC_training to acdc_test_set in each config.json

In [20]:
# running inference and ensemble on the test set is very similar to before

In [21]:
models, params_list, configs_list = prepare_models.prepare(["Deeplab_standard_example", "2D_Unet_standard_example"], "acdc_test_set")
model_a, model_b = models
test_loader = training_setup.prepare_test_loader(params_list[0], configs_list[0])
test_loader_2 = training_setup.prepare_test_loader(params_list[1], configs_list[1])

Constructing model:  Deeplab_standard_example
Number of classes:  4
Loading model from:  experiments/acdc_test_set/Deeplab_standard_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Constructing model:  2D_Unet_standard_example
Number of classes:  4
Loading model from:  experiments/acdc_test_set/2D_Unet_standard_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  17266436
Total number of trainable parameters of model:  17266436 

Dataset mean and std:  69.52275548950034 90.27520888722917
Using per slice normalization!


In [22]:
predictions_deeplab = inference.run_model_test_inference(test_loader, model_a, params_list[0], logsoft_preds=True)
predictions_unet = inference.run_model_test_inference(test_loader_2, model_b, params_list[1], logsoft_preds=True)

In [23]:
predictions_double = get_double_seg_predictions_test("Deeplab_standard_example", "Deeplab_ROI_example", "acdc_test_set")

Constructing model:  Deeplab_standard_example
Number of classes:  4
Loading model from:  experiments/acdc_test_set/Deeplab_standard_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Dataset mean and std:  69.52275548950034 90.27520888722917
Constructing model:  Deeplab_ROI_example
Number of classes:  4
Loading model from:  experiments/acdc_test_set/Deeplab_ROI_example/model_checkpoint.pt
Weights loaded successfully!
Total number of parameters of model:  2412094
Total number of trainable parameters of model:  2412094 

Dataset mean and std:  69.52275548950034 90.27520888722917


In [25]:
logsoftmax_volumes = [predictions_unet, predictions_deeplab, predictions_double]
combined_volumes = inference.ensemble_inference_test(logsoftmax_volumes)

In [26]:
# saving the results to a folder
save_complete_nii_results.save_predictions_test(combined_volumes, "ResNeXt_DeepLabV3_plus", "acdc_test_set")

Dataset mean and std:  69.52275548950034 90.27520888722917
Volume shape:  torch.Size([10, 480, 480])  and original shape:  [232, 256]
Saved volume shape, dtype and unique elems:  (232, 256, 10) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient101\patient101_frame01.nii.gz
Volume number  1  saved successfully!
Volume shape:  torch.Size([10, 480, 480])  and original shape:  [232, 256]
Saved volume shape, dtype and unique elems:  (232, 256, 10) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient101\patient101_frame14.nii.gz
Volume number  2  saved successfully!
Volume shape:  torch.Size([8, 480, 480])  and original shape:  [216, 256]
Saved volume shape, dtype and unique elems:  (216, 256, 8) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient102\patient102_frame01.nii.gz
Volume number  3  saved successfully!
Volume shape:  torch.Size([8, 480,

Saved volume shape, dtype and unique elems:  (256, 216, 9) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient118\patient118_frame10.nii.gz
Volume number  30  saved successfully!
Volume shape:  torch.Size([17, 480, 480])  and original shape:  [174, 208]
Saved volume shape, dtype and unique elems:  (174, 208, 17) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient119\patient119_frame01.nii.gz
Volume number  31  saved successfully!
Volume shape:  torch.Size([17, 480, 480])  and original shape:  [174, 208]
Saved volume shape, dtype and unique elems:  (174, 208, 17) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient119\patient119_frame09.nii.gz
Volume number  32  saved successfully!
Volume shape:  torch.Size([8, 480, 480])  and original shape:  [240, 166]
Saved volume shape, dtype and unique elems:  (240, 166, 8) uint8 [0 1 2 3]
Volume path:  C

Volume shape:  torch.Size([10, 480, 480])  and original shape:  [256, 216]
Saved volume shape, dtype and unique elems:  (256, 216, 10) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient136\patient136_frame12.nii.gz
Volume number  58  saved successfully!
Volume shape:  torch.Size([9, 480, 480])  and original shape:  [216, 256]
Saved volume shape, dtype and unique elems:  (216, 256, 9) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient137\patient137_frame01.nii.gz
Volume number  59  saved successfully!
Volume shape:  torch.Size([9, 480, 480])  and original shape:  [216, 256]
Saved volume shape, dtype and unique elems:  (216, 256, 9) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient137\patient137_frame11.nii.gz
Volume number  60  saved successfully!
Volume shape:  torch.Size([9, 480, 480])  and original shape:  [216, 256]
Saved volume shape

Volume shape:  torch.Size([10, 480, 480])  and original shape:  [216, 256]
Saved volume shape, dtype and unique elems:  (216, 256, 10) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient115\patient115_frame01.nii.gz
Volume number  85  saved successfully!
Volume shape:  torch.Size([10, 480, 480])  and original shape:  [216, 256]
Saved volume shape, dtype and unique elems:  (216, 256, 10) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient115\patient115_frame13.nii.gz
Volume number  86  saved successfully!
Volume shape:  torch.Size([9, 480, 480])  and original shape:  [214, 256]
Saved volume shape, dtype and unique elems:  (214, 256, 9) uint8 [0 1 2 3]
Volume path:  C:\Users\Dorel\Documents\GitHub\imATFIB\datasets\acdc_test_set\patient120\patient120_frame01.nii.gz
Volume number  87  saved successfully!
Volume shape:  torch.Size([9, 480, 480])  and original shape:  [214, 256]
Saved volume sha